<h3> This notebook is a follow up of plasma_test_final.ipynb and serum_test_final.ipynb</h3>

<h1 style="background-color:#ff5200;font-family:newtimeroman;font-size:350%;text-align:center;border-radius: 15px 50px;">Table Of Content</h1>



[1. Libraries And Utilities](#1)

[2 Processed Dataset from previous Notebook](#2)

[3 Models](#3)

[4. Visualization](#4)


<a id="1"></a>

<h1 style="background-color:#d8f8b7;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Libraries And Utilities</h1>

[Download DATASETS from here](https://github.com/utshabkg/LungCancer_Biomarker/tree/main/dataset)

In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import lazypredict
from lazypredict.Supervised import LazyClassifier

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import classification_report

from sklearn.model_selection import ShuffleSplit, train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, LabelEncoder

import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import RidgeClassifierCV
from yellowbrick.model_selection import FeatureImportances

<a id="2"></a>

<h1 style="background-color:#d8f8b7;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Processed Dataset from previous Notebook</h1>

[Download DATASETS from here](https://github.com/utshabkg/LungCancer_Biomarker/tree/main/dataset)

In [25]:
plasma = pd.read_csv('G:/Research/SM Sir/LungCancer_Biomarker/final and best approach/Approach 1/plasma(Specific Metabolics)_final.csv')
serum = pd.read_csv('G:/Research/SM Sir/LungCancer_Biomarker/final and best approach/Approach 1/serum(Specific Metabolics)_final.csv')
plasma.head()
plasma = plasma.drop('Unnamed: 0', axis=1)
serum = serum.drop('Unnamed: 0', axis=1)

In [26]:
plasma

,asparagine,benzoic acid,tryptophan,uric acid,alpha-ketoglutarate,citrulline,glutamine,malic acid,methionine sulfoxide,3-phosphoglycerate,...,aspartic acid,lactic acid,maltose,maltotriose,phenol,phosphoethanolamine,pyrophosphate,pyruvic acid,taurine,Class
0,-0.38,-1.31,0.52,-1.02,0.51,-0.33,-0.19,0.76,0.19,-0.49,...,0.02,-0.04,-0.34,-1.21,-0.10,0.03,-0.27,1.38,0.41,disease
1,0.95,-0.46,-0.90,-1.24,-0.39,0.69,1.04,0.09,-0.47,0.92,...,0.40,0.15,-0.29,-0.13,0.22,-0.83,-0.41,0.29,-0.16,disease
2,0.69,-0.49,0.83,-0.23,0.35,-0.56,0.36,0.06,-0.71,0.19,...,0.18,1.06,0.24,0.68,0.98,0.02,0.95,-0.19,0.83,disease
3,0.35,1.68,-1.11,-0.48,-0.75,-0.99,1.15,-0.41,-0.65,0.33,...,-0.35,0.53,-0.19,-0.38,0.69,-0.19,-0.59,-0.66,1.25,disease
4,0.30,-1.20,0.01,0.76,-0.14,0.77,-0.31,0.76,0.12,0.60,...,0.01,-0.33,-0.01,1.43,-0.27,0.32,0.52,-0.09,0.08,disease
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,0.77,-0.63,0.98,0.05,-0.30,0.08,1.06,-0.97,0.33,-0.89,...,-0.82,-0.72,-0.35,-0.38,-0.68,-0.45,-0.96,-0.12,-0.37,control
78,-0.43,-1.06,-0.03,1.56,-0.64,-0.41,-0.13,-0.67,-0.37,0.36,...,-0.73,0.64,-0.19,-0.60,0.79,-0.42,-0.22,-0.51,-0.23,control
79,-0.10,1.50,0.94,0.30,0.78,0.91,1.47,1.22,0.38,-0.47,...,-0.52,-0.53,-0.35,0.05,-0.44,-0.24,-0.67,-0.36,-0.54,control
80,-1.22,0.90,-1.34,-0.01,-0.14,-1.17,-1.39,-0.09,0.38,-0.78,...,-0.67,-0.46,-0.50,-0.83,-0.18,-0.44,-0.92,-0.10,-1.05,control


Keep labels as an numpy array.

In [27]:
labels = np.reshape(plasma.Class.to_numpy(), (82,1))
targets = plasma.iloc[:, -1].unique()

<a id="3"></a>

<h1 style="background-color:#d8f8b7;font-family:newtimeroman;font-size:250%;text-align:center;border-radius: 15px 50px;">Models</h1>

In [28]:
# !pip install lazypredict

In [29]:
def get_xy(df):
  X = df.drop('Class', axis=1)
  X = X.apply(pd.to_numeric)
  y = df['Class'].copy()
  y = y.map({"control": 0, "disease": 1})
  return X, y

In [30]:
# necessary functions
labels = np.reshape(plasma['Class'].to_numpy(), (82,1))

def pca_analysis(df, n_components):
    pca = PCA(n_components=n_components)
    pca_result = pca.fit_transform(df.iloc[:, :-1].values)

    pca_concatenate = np.concatenate([pca_result, labels], axis=1)
    return pca_concatenate

def make_dataframe(array, n_features):
    col = []
    for i in range(1, n_features+1):
        col.append(f'pc{i}')
    col.append('Class')
    return pd.DataFrame(array, columns = col)

def tsne_analysis(df, n_components):
    tsne = TSNE(n_components=n_components)
    tsne_result = tsne.fit_transform(df.iloc[:, :-1].values)

    tsne_concatenate = np.concatenate([tsne_result, labels], axis=1)
    return tsne_concatenate
def tsne_dataframe(array, n_features):
    col = []
    for i in range(1, n_features+1):
        col.append(f'tsne{i}')
    col.append('Class')
    return pd.DataFrame(array, columns = col)

def get_xy(df):
  X = df.drop('Class', axis=1)
  X = X.apply(pd.to_numeric)
  y = df['Class'].copy()
  y = y.map({"control": 0, "disease": 1})
  return X, y

In [34]:
accuracy = []
for k in [0, 2, 3, 4, 5]:
    
    if k==0:
        name = ['Plasma(Raw)',  'Plasma(Exploratory)',]
        dfs = [plasma]
    
    else:
        name = [f'pca{k}_plasma(Exploratory)']
        vars()[f'pca{k}_plasma'] = pca_analysis(plasma, k)
        vars()[f'pca{k}_plasma_df'] = make_dataframe(vars()[f'pca{k}_plasma'], k)
        dfs = [vars()[f'pca{k}_plasma_df']]
    
    random_state = 42

    for idx, data in enumerate(dfs):
        result = []
        if data.columns[-1]=='Class':
            X = data.drop('Class', axis=1)
        else:
            X = data
        y = plasma['Class'].copy()
        y = y.map({"control": 0, "disease": 1})
        rs = ShuffleSplit(n_splits=10, test_size=.2, random_state=random_state)

        for train_index, test_index in rs.split(X):
            clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=matthews_corrcoef, predictions=True)
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            scaler = RobustScaler()

            cols = X_train.columns
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            X_train = pd.DataFrame(X_train, columns=[cols])
            X_test = pd.DataFrame(X_test, columns=[cols])

            models, predictions = clf.fit(X_train, X_test, y_train, y_test)
            result.append(models)
        #     print(result)

        acc = []
        for i in range(1):
           acc.append(result[i].loc['RidgeClassifier']['Balanced Accuracy'])
#                     print(result[i].index[j], result[i].iloc[j]['Balanced Accuracy'])

        print(f'\n{name[idx]}: {sum(acc)/len(acc)}')
        accuracy.append([f'{name[idx]}', f'{sum(acc)/len(acc)}'])

accuracy_df1 = pd.DataFrame(accuracy, columns=['Method', 'Accuracy'])
accuracy_df1

  7%|█████▋                                                                             | 2/29 [00:00<00:01, 18.35it/s]


Plasma(Raw): 0.9090909090909092


  7%|█████▋                                                                             | 2/29 [00:00<00:01, 18.52it/s]


pca2_plasma(Exploratory): 0.8181818181818181


  7%|█████▋                                                                             | 2/29 [00:00<00:01, 17.24it/s]

...

  3%|██▊                                                                                | 1/29 [00:00<00:03,  8.13it/s]


pca4_plasma(Exploratory): 0.8181818181818181


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 35.37it/s]


pca5_plasma(Exploratory): 0.8181818181818181


,Method,Accuracy
0,Plasma(Raw),0.9090909090909092
1,pca2_plasma(Exploratory),0.8181818181818181
2,pca3_plasma(Exploratory),0.8181818181818181
3,pca4_plasma(Exploratory),0.8181818181818181
4,pca5_plasma(Exploratory),0.8181818181818181


In [35]:
accuracy = []
for k in [0, 2, 3, 4, 5]:
    
    if k==0:
        name = ['Serum(Raw)', 'Serum(Exploratory)']
        dfs = [serum]
    
    else:
        name = [f'pca{k}_serum(Exploratory)']
        
        vars()[f'pca{k}_serum'] = pca_analysis(serum, k)
        vars()[f'pca{k}_serum_df'] = make_dataframe(vars()[f'pca{k}_serum'], k)
        
        dfs = [vars()[f'pca{k}_serum_df']]
    
    random_state = 42

    for idx, data in enumerate(dfs):
        result = []
        if data.columns[-1]=='Class':
            X = data.drop('Class', axis=1)
        else:
            X = data
        y = plasma['Class'].copy()
        y = y.map({"control": 0, "disease": 1})
        rs = ShuffleSplit(n_splits=10, test_size=.2, random_state=random_state)

        for train_index, test_index in rs.split(X):
            clf = LazyClassifier(verbose=0, ignore_warnings=True, custom_metric=matthews_corrcoef, predictions=True)
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]

            scaler = RobustScaler()

            cols = X_train.columns
            X_train = scaler.fit_transform(X_train)
            X_test = scaler.transform(X_test)

            X_train = pd.DataFrame(X_train, columns=[cols])
            X_test = pd.DataFrame(X_test, columns=[cols])

            models, predictions = clf.fit(X_train, X_test, y_train, y_test)
            result.append(models)
        #     print(result)

        acc = []
        for i in range(1):
           acc.append(result[i].loc['RandomForestClassifier']['Balanced Accuracy'])
#                     print(result[i].index[j], result[i].iloc[j]['Balanced Accuracy'])

        print(f'\n{name[idx]}: {sum(acc)/len(acc)}')
        accuracy.append([f'{name[idx]}', f'{sum(acc)/len(acc)}'])

accuracy_df2 = pd.DataFrame(accuracy, columns=['Method', 'Accuracy'])
accuracy_df2

  7%|█████▋                                                                             | 2/29 [00:00<00:01, 17.70it/s]


Serum(Raw): 0.7803030303030303


  7%|█████▋                                                                             | 2/29 [00:00<00:01, 18.02it/s]


pca2_serum(Exploratory): 0.7878787878787878


  7%|█████▋                                                                             | 2/29 [00:00<00:01, 17.70it/s]

...

  7%|█████▋                                                                             | 2/29 [00:00<00:01, 17.09it/s]


pca4_serum(Exploratory): 0.7803030303030303


100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:00<00:00, 40.45it/s]


pca5_serum(Exploratory): 0.7878787878787878


,Method,Accuracy
0,Serum(Raw),0.7803030303030303
1,pca2_serum(Exploratory),0.7878787878787878
2,pca3_serum(Exploratory),0.8636363636363636
3,pca4_serum(Exploratory),0.7803030303030303
4,pca5_serum(Exploratory),0.7878787878787878


In [37]:
accuracy_df1.to_csv('G:/Research/SM Sir/LungCancer_Biomarker/visualizations/curve/ours_plasma.csv', index=False)
accuracy_df2.to_csv('G:/Research/SM Sir/LungCancer_Biomarker/visualizations/curve/ours_serum.csv', index=False)